<a href="https://colab.research.google.com/github/JoshDumo/QuantumChem/blob/main/Hydrogen_GroundState.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Quantum Chemistry Using Qiskit Nature



In [1]:
!pip install qiskit
!pip install qiskit_nature

     |████████████████████████████████| 6.0MB 3.8MB/s 
     |████████████████████████████████| 17.9MB 208kB/s 
     |████████████████████████████████| 204kB 51.6MB/s 
     |████████████████████████████████| 215kB 55.1MB/s 
     |████████████████████████████████| 2.1MB 32.9MB/s 
     |████████████████████████████████| 1.0MB 40.4MB/s 
     |████████████████████████████████| 51kB 5.0MB/s 
     |████████████████████████████████| 194kB 55.5MB/s 
     |████████████████████████████████| 81kB 8.1MB/s 
     |████████████████████████████████| 614kB 50.8MB/s 
     |████████████████████████████████| 3.2MB 49.8MB/s 
     |████████████████████████████████| 5.5MB 43.6MB/s 
  Created wheel for qiskit: filename=qiskit-0.25.1-cp37-none-any.whl size=3064 sha256=ae19ab5f6e3004742a9426c600ecd6778de702003d2246f85b239b70bb74a4e3
  Stored in directory: /root/.cache/pip/wheels/e2/00/23/7328e187ec5202e637b2fb6b2a0c68029758c90abded1eee60
  Created wheel for python-constraint: filename=python_constraint-1.4.0-py2

## Installing PySCF

In [2]:
!pip install pyscf

     |████████████████████████████████| 29.7MB 158kB/s 


## Imports

In [3]:
from qiskit_nature.drivers import PySCFDriver, UnitsType, Molecule
from qiskit_nature.problems.second_quantization.electronic import ElectronicStructureProblem



## Preparing the molecule

In [4]:
molecule = Molecule(geometry=[['H', [0., 0., 0.]],
                              ['H', [0., 0., 0.735]]],
                     charge=0, multiplicity=1)

driver = PySCFDriver(molecule = molecule, unit=UnitsType.ANGSTROM, basis='sto3g')
problem = ElectronicStructureProblem(driver)



## Generating the second-quantized operators

In [5]:

second_q_ops = problem.second_q_ops()
main_op = second_q_ops[0]

num_particles = (problem.molecule_data_transformed.num_alpha,
                 problem.molecule_data_transformed.num_beta)

num_spin_orbitals = 2 * problem.molecule_data.num_molecular_orbitals


## Getting the optimizer

In [6]:
from qiskit.algorithms.optimizers import L_BFGS_B

optimizer = L_BFGS_B()

## Doing the mapping

In [7]:
from qiskit_nature.mappers.second_quantization import ParityMapper
from qiskit_nature.converters.second_quantization import QubitConverter

mapper = ParityMapper()
converter = QubitConverter(mapper=mapper, two_qubit_reduction=True)

# map to qubit operators
qubit_op = converter.convert(main_op, num_particles=num_particles)

## Preparing the initial state ansatz

In [8]:
from qiskit_nature.circuit.library import HartreeFock

init_state = HartreeFock(num_spin_orbitals, num_particles, converter)

In [11]:
from qiskit.circuit.library import TwoLocal
%matplotlib inline

ansatz = TwoLocal(num_spin_orbitals, ['ry', 'rz'], 'cz')

ansatz.draw()
# add the initial state
ansatz.compose(init_state, front=True)

## Setting the backend for the quantum computation

In [12]:

from qiskit import Aer

backend = Aer.get_backend('statevector_simulator')

# setup and run VQE
from qiskit.algorithms import VQE

algorithm = VQE(ansatz,
                optimizer=optimizer,
                quantum_instance=backend)

result = algorithm.compute_minimum_eigenvalue(qubit_op)
print(result.eigenvalue.real)

electronic_structure_result = problem.interpret(result)
print(electronic_structure_result)

-1.8572750298409044
=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -1.857275029841
  - computed part:      -1.857275029841
~ Nuclear repulsion energy (Hartree): 0.719968994449
> Total ground state energy (Hartree): -1.137306035392
 
=== MEASURED OBSERVABLES ===
 
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  1.3889487]
 
